In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [2]:
import os
from os import listdir
from os.path import isfile, join
from itertools import islice
from skimage.external import tifffile
from skimage.external.tifffile import imread
import ipywidgets as widgets

In [3]:
bigDir = r'Z:\CookLab\Liu\20190816_organizedData_MCM_loading\20200829_h3k9me3'

In [9]:
file = f"{bigDir}\\200819_cellinfo_v7_20.csv"
data = pd.read_csv(file)

In [5]:
data.head()

,Unnamed: 0,Unnamed: 0.1,file,original cell name,movie,x,y,ab1 (MCM),ab1 ch,ab2 (heterochromatin),...,outer_DAPI_eu,nuc_h3k9me3_total,inner_h3k9me3_total,outer_h3k9me3_total,nuc_h3k9me3_het,inner_h3k9me3_het,outer_h3k9me3_het,nuc_h3k9me3_eu,inner_h3k9me3_eu,outer_h3k9me3_eu
0,0,0,20200819_cell2_01.ciz,20200818xy19_6.czi,200819xy19,1526.899,85.295,H3k9me3,1,MCM2,...,8.056507e+08,1.556340e+09,507823395.0,1.048516e+09,658184045.0,359266210.0,298917835.0,8.981557e+08,148557185.0,749598468.0
1,1,1,20200819_cell2_02.ciz,20200818xy19_8.czi,200819xy19,1112.568,94.321,H3k9me3,1,MCM2,...,8.659832e+08,1.519626e+09,374000205.0,1.145626e+09,603664320.0,217624195.0,386040125.0,9.159621e+08,156376010.0,759586103.0
2,2,2,20200819_cell2_03.ciz,20200818xy19_22.czi,200819xy19,1412.269,290.826,H3k9me3,1,MCM2,...,9.180782e+08,1.031510e+09,329936617.0,7.015734e+08,457591039.0,256291447.0,201299592.0,5.739190e+08,73645170.0,500273850.0
3,3,3,20200819_cell2_04.ciz,20200818xy19_25.czi,200819xy19,1764.520,336.100,H3k9me3,1,MCM2,...,1.037998e+09,1.255900e+09,244456731.0,1.011443e+09,579531132.0,203671825.0,375859307.0,6.763688e+08,40784906.0,635583861.0
4,4,4,20200819_cell2_05.ciz,20200818xy19_27.czi,200819xy19,188.411,367.506,H3k9me3,1,MCM2,...,1.771447e+09,1.834223e+09,612423518.0,1.221800e+09,716771839.0,397258618.0,319513221.0,1.117451e+09,215164900.0,902286564.0


In [4]:
file2 = f"{bigDir}\\200819_cellinfo_v6_28.csv"
data2 = pd.read_csv(file2)

In [5]:
file3 = f"{bigDir}\\200819_cellinfo_v6_29.csv"
data3 = pd.read_csv(file3)

# Questions to answer
1. Do the cells change the volume of nuclei in MgCL2? - answered by Liu
2. What is the distribution of heterochromatin intensity threshold in ctrl population of cells?
3. Compare total heterochromatin marker signal in cells in ctl and MgCl2 groups. which group has more signal?
4. Find thresdhold for heterochromatin volume in MgCl2 treated cells that will match the intensity threshold from ctl cells. 

In [6]:
myHetSignalFinal=[]
myChannelFinal=[]
myHetSignal_1Final=[]

In [9]:
data_test=data

In [8]:
%matplotlib notebook
hp1=0

## calculating 20% brightest pixles
het_percentile=28

# Progress Bar
i=60

progBar=widgets.IntProgress(
    value=i,
    min=0,
    max=len(data2),
    step=1,
    description='Progress:',
    orientation='horizontal'
)
display(progBar)
for i,myCell in islice(data2.iterrows(), i,None):
    if myCell.group=='Mgcl2':
        ## create folder to store this percentile data if it doesn't exist yet
        newSegFolder = os.path.dirname(myCell.path).replace('data_tiff',f'segmentation_{het_percentile}')
        newResultsFolder = os.path.dirname(myCell.path).replace('data_tiff', 'results')

        try:
            os.mkdir(newSegFolder)
        except:
            pass

        try:
            os.mkdir(newResultsFolder)
        except:
            pass


        ## open tiff image of a cell
        try:
            myImage=imread(myCell.path)    
        except:
            myCell.path=myCell.path.replace('.tif','.tiff')
            myImage=imread(myCell.path)    
        #print(myImage.shape)
        ## opening the channel according to the dataset
        myChannel = myImage[:, hp1, :, :] 

        ## open nucleus mask
        ### if the file is not found in the directory, trying to change the file extension 
        #### this is because there are some file files that are saved as tiff image 
        nucleusMaskPath=myCell.path
        nucleusMaskPath=nucleusMaskPath.replace('data_tiff','segmentation_nucleus_Otsu')
        nucleusMaskPath=nucleusMaskPath.replace('.tif','_nucleus.tif')
        try:
            nucleusMask=imread(nucleusMaskPath)
        except:
            nucleusMaskPath=nucleusMaskPath.replace('.tiff','.tif')
            nucleusMask=imread(nucleusMaskPath)


        ## make nucleus mask binary
        nucleusMask = nucleusMask.astype(bool)

        ## clean image
        myNucleus = myChannel[nucleusMask]
        #myChannelFinal_mgcl.extend(myNucleus)
        #print(myNucleus.shape)
        #sns.distplot(myNucleus)

        ## find threshold
        myPercentile = np.nanpercentile(myNucleus, (100-het_percentile)) 
        #plt.axvline(myPercentile, 0,1)


        ## threshold
        myHetMask=np.zeros(myChannel.shape).astype('bool')
        myHetMask[myChannel >= myPercentile] = 1

        ## clean from elements outside the nucleus
        cleanNucleusMask=~nucleusMask
        myHetMask[cleanNucleusMask]=0
        cleanHetMask = ~myHetMask

        ## save segmentation mask
        maskFile = os.path.basename(myCell.path).replace('.tif',f'_hetChrom.tif')
        tifffile.imsave(os.path.join(newSegFolder,maskFile), myHetMask.astype('uint8'))
        
        myChannel[~myHetMask]=0
        myHetSignal_1=myChannel.ravel()
        myHetSignal_1=[i for i in myHetSignal_1 if i !=0]
        
        data2.loc[i,'hetThreshold'] = min(myHetSignal_1)
        data2.loc[i,'volThreshold'] = myPercentile
        
        #sns.distplot(myHetSignal)
        progBar.value=progBar.value+1

    

  

IntProgress(value=60, description='Progress:', max=113)

In [11]:
hetThresholdCTL=data.loc[(['CTL' in x for x in data.group]), 'hetThreshold']

In [12]:
volThresholdCTL=data.loc[(['CTL' in x for x in data.group]), 'volThreshold']

In [14]:
data2.loc[(['CTL' in x for x in data2.group]), 'hetThreshold']=hetThresholdCTL

In [15]:
data2.loc[(['CTL' in x for x in data2.group]), 'volThreshold']=volThresholdCTL

In [17]:
hetThresholdMGCL_28=data2.loc[(['Mgcl2' in x for x in data2.group]), 'hetThreshold']
volThresholdMGCL_28=data2.loc[(['Mgcl2' in x for x in data2.group]), 'volThreshold']

In [18]:
%matplotlib notebook
hp1=0

## calculating 20% brightest pixles
het_percentile=29

# Progress Bar
i=60

progBar=widgets.IntProgress(
    value=i,
    min=0,
    max=len(data3),
    step=1,
    description='Progress:',
    orientation='horizontal'
)
display(progBar)
for i,myCell in islice(data3.iterrows(), i,None):
    if myCell.group=='Mgcl2':
        ## create folder to store this percentile data if it doesn't exist yet
        newSegFolder = os.path.dirname(myCell.path).replace('data_tiff',f'segmentation_{het_percentile}')
        newResultsFolder = os.path.dirname(myCell.path).replace('data_tiff', 'results')

        try:
            os.mkdir(newSegFolder)
        except:
            pass

        try:
            os.mkdir(newResultsFolder)
        except:
            pass


        ## open tiff image of a cell
        try:
            myImage=imread(myCell.path)    
        except:
            myCell.path=myCell.path.replace('.tif','.tiff')
            myImage=imread(myCell.path)    
        #print(myImage.shape)
        ## opening the channel according to the dataset
        myChannel = myImage[:, hp1, :, :] 

        ## open nucleus mask
        ### if the file is not found in the directory, trying to change the file extension 
        #### this is because there are some file files that are saved as tiff image 
        nucleusMaskPath=myCell.path
        nucleusMaskPath=nucleusMaskPath.replace('data_tiff','segmentation_nucleus_Otsu')
        nucleusMaskPath=nucleusMaskPath.replace('.tif','_nucleus.tif')
        try:
            nucleusMask=imread(nucleusMaskPath)
        except:
            nucleusMaskPath=nucleusMaskPath.replace('.tiff','.tif')
            nucleusMask=imread(nucleusMaskPath)


        ## make nucleus mask binary
        nucleusMask = nucleusMask.astype(bool)

        ## clean image
        myNucleus = myChannel[nucleusMask]
        #myChannelFinal_mgcl.extend(myNucleus)
        #print(myNucleus.shape)
        #sns.distplot(myNucleus)

        ## find threshold
        myPercentile = np.nanpercentile(myNucleus, (100-het_percentile)) 
        #plt.axvline(myPercentile, 0,1)


        ## threshold
        myHetMask=np.zeros(myChannel.shape).astype('bool')
        myHetMask[myChannel >= myPercentile] = 1

        ## clean from elements outside the nucleus
        cleanNucleusMask=~nucleusMask
        myHetMask[cleanNucleusMask]=0
        cleanHetMask = ~myHetMask

        ## save segmentation mask
        maskFile = os.path.basename(myCell.path).replace('.tif',f'_hetChrom.tif')
        tifffile.imsave(os.path.join(newSegFolder,maskFile), myHetMask.astype('uint8'))
        
        myChannel[~myHetMask]=0
        myHetSignal_1=myChannel.ravel()
        myHetSignal_1=[i for i in myHetSignal_1 if i !=0]
        
        data3.loc[i,'hetThreshold'] = min(myHetSignal_1)
        data3.loc[i,'volThreshold'] = myPercentile
        
        #sns.distplot(myHetSignal)
        progBar.value=progBar.value+1

    

  

IntProgress(value=60, description='Progress:', max=113)

In [20]:
hetThresholdMGCL_29=data3.loc[(['Mgcl2' in x for x in data3.group]), 'hetThreshold']
volThresholdMGCL_29=data3.loc[(['Mgcl2' in x for x in data3.group]), 'volThreshold']

In [21]:
data3.loc[(['CTL' in x for x in data3.group]), 'hetThreshold']=hetThresholdCTL
data3.loc[(['CTL' in x for x in data3.group]), 'volThreshold']=volThresholdCTL

In [22]:
data2.to_csv(file2.replace("v6","v7"))

In [23]:
data3.to_csv(file3.replace("v6","v7"))

In [24]:
np.mean(hetThresholdMGCL_29)

10143.905660377359

In [25]:
np.mean(volThresholdMGCL_29)

10143.84962264151

In [26]:
np.median(hetThresholdMGCL_29)

9592.0

In [27]:
np.median(hetThresholdMGCL_28)

9878.0

In [28]:
np.median(hetThresholdCTL)

9686.5

In [31]:
stats.ttest_ind(hetThresholdCTL,hetThresholdMGCL_28)

Ttest_indResult(statistic=-0.30664652647024276, pvalue=0.7596869157589916)

In [32]:
stats.ttest_ind(hetThresholdCTL,hetThresholdMGCL_29)

Ttest_indResult(statistic=0.16462354994115627, pvalue=0.8695397086530996)